In [117]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_test['Transported'] = 'Test'
test_id = df_test['PassengerId']

In [118]:
df_all = pd.concat([df_train, df_test])
df_all = df_all.drop('PassengerId', axis = 1)

In [119]:
df_all.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [120]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12970 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12682 non-null  object 
 1   CryoSleep     12660 non-null  object 
 2   Cabin         12671 non-null  object 
 3   Destination   12696 non-null  object 
 4   Age           12700 non-null  float64
 5   VIP           12674 non-null  object 
 6   RoomService   12707 non-null  float64
 7   FoodCourt     12681 non-null  float64
 8   ShoppingMall  12664 non-null  float64
 9   Spa           12686 non-null  float64
 10  VRDeck        12702 non-null  float64
 11  Name          12676 non-null  object 
 12  Transported   12970 non-null  object 
dtypes: float64(6), object(7)
memory usage: 1.4+ MB


In [121]:
df_all['Name'].fillna(0, inplace=True)
df_all['Name'] = df_all['Name'].apply(lambda x: 'known' if x!=0 else 'Unknown')

In [122]:
df_all['VIP'].fillna(False, inplace=True)
df_all['VIP'] = df_all['VIP'].astype(float)
df_all['VRDeck'].fillna(0, inplace=True)
df_all['Spa'].fillna(0, inplace=True)
df_all['ShoppingMall'].fillna(0, inplace=True)
df_all['FoodCourt'].fillna(0, inplace=True)
df_all['RoomService'].fillna(0, inplace=True)

In [123]:
df_all['Age'].fillna('Unknown', inplace=True)
df_all['Destination'].fillna('Unknown', inplace=True)
df_all['CryoSleep'].fillna(False, inplace=True)
df_all['CryoSleep'] = df_all['CryoSleep'].astype(float)
df_all['HomePlanet'].fillna('Unknown', inplace=True)
df_all = df_all.drop(['Cabin', 'Age', 'HomePlanet', 'Destination', 'Name'], axis = 1)

In [124]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df_val = df_all[df_all['Transported'] == 'Test']
df_train = df_all[df_all['Transported'] != 'Test']
X = df_train.drop('Transported', axis = 1)
y = df_train['Transported'].astype(bool)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_val = df_val.drop('Transported', axis = 1)

In [125]:
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7782888684452622


In [129]:
prediction = model.predict(X_val)
test_id = pd.DataFrame(test_id)
prediction = pd.DataFrame(prediction, 'Transported')
#result = test_id.join(prediction)
prediction

,0
0,True
1,False
2,True
3,True
4,True
...,...
4272,True
4273,False
4274,True
4275,True
